In [1]:
import base64
from openai import OpenAI
from transformers import AutoModelForVision2Seq, AutoTokenizer
from huggingface_hub import login
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
import torch

In [ ]:
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Code calls itself now,  \nLayers fold into layers,  \nInfinite loops churn.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [4]:
print(completion.choices[0].message.content)

Code calls itself now,  
Layers fold into layers,  
Infinite loops churn.


## image detection

In [11]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../image_detection/images_test/test/01.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "To which of the following categories does this stamp belong (just say the exact theme)? 'Historical Figures', 'Countries and Cities', 'Events', 'Art and Culture', 'Science and Technology', 'Nature', 'Sports', 'Symbols and Emblems', 'Special Interests', 'Miscellaneous'",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Historical Figures', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))


# Llama

In [ ]:
# read token form tokens.json file
import json


with open("tokens.json") as f:
    tokens = json.load(f)
    

login(token=tokens["llama"])

# model_name = "meta-llama/Llama-3.2-11B-Vision"
# model = AutoModelForVision2Seq.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)


In [12]:
# Load the image 
image_path = "../image_detection/images_test/test/A1-s.jpg"
image = Image.open(image_path)

In [3]:
model_id = "meta-llama/Llama-3.2-11B-Vision"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

C:\Users\Robin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Robin\.cache\huggingface\hub\models--meta-llama--Llama-3.2-11B-Vision. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [13]:
# Prepare the image for the model
prompt = "<|image|>Describe the image."
inputs = processor(image, prompt, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_new_tokens=100)
print(processor.decode(output[0]))


<|begin_of_text|><|image|><|begin_of_text|>Describe the image. I'm not able to identify the person in this image. I'm not able to identify the person in this image. I'm not able to identify the person in this image. I'm not able to identify the person in this image. I'm not able to identify the person in this image. I'm not able to identify the person in this image. I'm not able to identify the person in this image. I'm not able to identify the person in this image. I'm not able
